In [1]:
import tabula
import pandas as pd
import numpy as np

In [2]:
# Replace 'bank_statement.pdf' with the actual path to your PDF file
pdf_file_path = "/home/arnold/Desktop/test/xxxxx/MPESA_Statement.pdf"

In [3]:
def data_cleaning(data_frame):
    # Dropping the unnecessary columns
    data_frame = data_frame.drop(data_frame.columns[7:], axis=1)
    data_frame = data_frame.drop(data_frame.columns[[3]], axis=1)
    #Dropping NA rows in the receipt no. column
    data_frame = data_frame.dropna(subset="Receipt No.")
    #converting completion time to date object
    data_frame["Completion Time"] = pd.to_datetime(data_frame["Completion Time"]).dt.normalize()
    #replacing \r in the details column with space
    data_frame["Details"] =  data_frame["Details"].str.replace(r"\r"," ",regex=True)
    #replacing the minus sign in the withdrawn column
    data_frame["Withdrawn"] = data_frame["Withdrawn"].str.replace("-","")
    cols=["Paid In","Withdrawn","Balance"]
    data_frame[cols]= data_frame[cols].replace(",","",regex=True)
    #converting NAN to zero
    data_frame= data_frame.fillna(0)
    data_frame[cols] = data_frame[cols].apply(pd.to_numeric,axis=1)
    data_frame = data_frame.reindex(index=data_frame.index[::-1])

    return data_frame

In [4]:
def extract_tables_from_pdf(pdf_file):
    # Read the tables from the PDF file
    tables = tabula.read_pdf(pdf_file, pages="all", multiple_tables=True)

    # Combine all tables except the first one into a single DataFrame
    # We start at table 2 to ignore the summary tables
    combined_df = pd.concat(tables[2:], ignore_index=True)
    combined_df = data_cleaning(combined_df)
    return combined_df

In [ ]:
combined_data_frame = extract_tables_from_pdf(pdf_file_path)
combined_data_frame.head()

In [6]:
combined_data_frame.dtypes

Receipt No.                object
Completion Time    datetime64[ns]
Details                    object
Paid In                   float64
Withdrawn                 float64
Balance                   float64
dtype: object

In [7]:
#Categorisation of transactions
categories = {
    "Income":["Funds received from ","Deposit of Funds at Agent Till "],
    "Contra entry" : ["Business Payment from "],
    "Loan" : ["OverDraft of Credit Party","KCB M-PESA Withdraw","M-Shwari Withdraw","M-Shwari Loan Disburse"],
    "Airtime" : ["Airtime Purchase ","Airtime Purchase"],
    "Bundles" : ["Buy Bundles "],
    "Loan repayment" : ["OD Loan Repayment to 232323 - M-PESA Overdraw","M-Shwari Loan Repayment"],
    "Transaction charges" : ["Charge"],
    "Expenses" : ["Pay Bill to ","Pay Bill Online to ","Pay Bill Fuliza ","Customer Payment to Small Business to ",
                  "Customer Transfer Fuliza MPesa to ","Merchant Payment to ","Customer Transfer to "],
    "Agent withdrawals" : ["Customer Withdrawal At Agent Till "]
}


In [8]:
def transaction_categorisation(df,category):
    words = []
    mapping_dict = {}

    for i,j in category.items():
        for item in j:
            words.append(item)
            mapping_dict[item] = i

    joined_text = df["Details"].str.extract(f"({'|'.join(words)})")
    text_category = joined_text[0].map(mapping_dict)
    text_category = text_category.fillna("Others")
    return text_category

In [ ]:
transaction_categories = transaction_categorisation(combined_data_frame,categories)
combined_data_frame["Transaction Categories"]=transaction_categories

#combined_data_frame.insert(3,"Transaction Categories",transaction_categories)
combined_data_frame.head()

In [10]:
#Calculation of DTI
total_deposits = combined_data_frame["Paid In"].sum()
total_withdrawals = combined_data_frame["Withdrawn"].sum()
contra_entry_credits = combined_data_frame.loc[combined_data_frame["Transaction Categories"] =="Contra entry","Paid In"].sum()
contra_entry_debits = combined_data_frame.loc[combined_data_frame["Transaction Categories"] =="Contra entry","Withdrawn"].sum()
net_amount = total_deposits - contra_entry_credits
average_monthly_credits = net_amount/((combined_data_frame["Completion Time"].iloc[0] - combined_data_frame["Completion Time"].iloc[-1])/np.timedelta64("1","M"))
business_dti = average_monthly_credits * 0.10

In [41]:
# #Creating dataframe of dates and balances
def calculate_daily_balances(dataframe):

    unique_dates = dataframe["Completion Time"].unique()
    daily_dates = pd.date_range(start=dataframe["Completion Time"].min(),
                                 end=dataframe["Completion Time"].max(),freq="D")

    cols = ["Completion Time","Balance"]
    daily_balances = dataframe[cols].copy()
    daily_balances = daily_balances.drop_duplicates("Completion Time",keep="last")
    daily_balances = daily_balances.set_index("Completion Time")

    if len(unique_dates) != len(daily_dates):
        daily_balances = daily_balances.resample("D").asfreq().fillna(method="ffill")
        return daily_balances

    else:
        return daily_balances

In [44]:
latest_running_balance = calculate_daily_balances(combined_data_frame)
latest_running_balance.head()

,Balance
Completion Time,
2022-12-13,6994.33
2022-12-14,18098.33
2022-12-15,18098.33
2022-12-16,18098.33
2022-12-17,18098.33


In [61]:
monthly_transactions = combined_data_frame.resample(rule="M",on="Completion Time").sum()
monthly_transactions = monthly_transactions.drop("Balance",axis=1)
monthly_transactions

,Paid In,Withdrawn
Completion Time,,
2022-12-31,965150.0,1006035.56
2023-01-31,353990.0,356763.96
2023-02-28,232300.0,233747.28
2023-03-31,737820.0,737892.70
2023-04-30,507730.0,508218.38
2023-05-31,379730.0,364681.00
2023-06-30,166666.0,181832.36
